In [1]:
%load_ext autoreload
%autoreload 2

In [56]:
df_transformed = pd.read_pickle("/Users/juliaschmitt/code/JulSchmitt/vincentvanbot/raw_data/flat_resized_images_100.pkl")
df_transformed

,0,1,2,3,4,5,6,7,8,9,...,29990,29991,29992,29993,29994,29995,29996,29997,29998,29999
0,0.180392,0.168627,0.101961,0.262745,0.231373,0.180392,0.243137,0.196078,0.149020,0.184314,...,0.121569,0.149020,0.078431,0.062745,0.164706,0.098039,0.066667,0.149020,0.094118,0.058824
1,0.247059,0.239216,0.188235,0.180392,0.211765,0.160784,0.184314,0.231373,0.192157,0.172549,...,0.188235,0.403922,0.309804,0.207843,0.454902,0.364706,0.243137,0.435294,0.345098,0.223529
10,0.117647,0.086275,0.074510,0.760784,0.772549,0.705882,0.066667,0.050980,0.015686,0.070588,...,0.000000,0.086275,0.015686,0.070588,0.231373,0.149020,0.043137,0.133333,0.039216,0.031373
100,0.211765,0.207843,0.200000,0.219608,0.215686,0.207843,0.200000,0.188235,0.160784,0.223529,...,0.341176,0.419608,0.376471,0.368627,0.474510,0.419608,0.407843,0.431373,0.376471,0.364706
101,0.211765,0.164706,0.109804,0.196078,0.149020,0.094118,0.196078,0.149020,0.094118,0.207843,...,0.109804,0.113725,0.113725,0.105882,0.137255,0.137255,0.129412,0.235294,0.215686,0.192157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.364706,0.313726,0.278431,0.278431,0.235294,0.211765,0.298039,0.270588,0.239216,0.270588,...,0.619608,0.882353,0.839216,0.752941,0.831373,0.788235,0.701961,0.752941,0.705882,0.619608
96,0.443137,0.439216,0.423529,0.482353,0.482353,0.474510,0.396078,0.376471,0.352941,0.403922,...,0.188235,0.443137,0.403922,0.356863,0.486275,0.462745,0.415686,0.466667,0.439216,0.364706
97,0.227451,0.203922,0.203922,0.270588,0.247059,0.247059,0.231373,0.211765,0.196078,0.247059,...,0.568627,0.486275,0.458824,0.427451,0.509804,0.482353,0.450980,0.627451,0.623529,0.603922
98,0.333333,0.341176,0.337255,0.337255,0.337255,0.368627,0.345098,0.364706,0.380392,0.200000,...,0.592157,0.678431,0.674510,0.592157,0.639216,0.654902,0.588235,0.584314,0.627451,0.509804


In [4]:
#X = np.array(df_transformed)

In [54]:
img = np.array(df_transformed.iloc[:1])
img
#img.reshape(100, 100, 3).shape

array([[0.18039216, 0.16862746, 0.10196079, ..., 0.14901961, 0.09411765,
        0.05882353]], dtype=float32)

In [ ]:
#knn_model = NearestNeighbors(n_neighbors=20).fit(df_transformed)

In [ ]:
ind_list = list(knn_model.kneighbors(img,n_neighbors=20)[1][0])[1:] # get n closest points, unpack indices to a list
df_closest_images = df.iloc[ind_list, :]
df_closest_images.head()

In [71]:
df = get_data_locally(32000)
df

,AUTHOR,BORN-DIED,TITLE,DATE,TECHNIQUE,LOCATION,URL,FORM,TYPE,SCHOOL,TIMEFRAME
0,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)",Venus and Adonis,1574-88,"Oil on canvas, 68 x 95 cm","Fogg Art Museum, Harvard University, Cambridge",https://www.wga.hu/art/a/aachen/adonis.jpg,painting,mythological,German,1601-1650
1,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)",Allegory,1598,"Oil on copper, 56 x 47 cm","Alte Pinakothek, Munich",https://www.wga.hu/art/a/aachen/allegory.jpg,painting,mythological,German,1601-1650
2,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)","Allegory of Peace, Art and Abundance",1602,"Oil on canvas, 197 x 142 cm","The Hermitage, St. Petersburg",https://www.wga.hu/art/a/aachen/allegorz.jpg,painting,mythological,German,1601-1650
3,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)","Jupiter, Antiope and Cupid",1595-98,"Oil on copper, 31 x 21 cm","Kunsthistorisches Museum, Vienna",https://www.wga.hu/art/a/aachen/antiope.jpg,painting,mythological,German,1601-1650
4,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)","Pallas Athena, Venus and Juno",1593,"Oil on canvas, 54 x 67 cm","Museum of Fine Arts, Boston",https://www.wga.hu/art/a/aachen/athena.jpg,painting,mythological,German,1601-1650
...,...,...,...,...,...,...,...,...,...,...,...
50551,"ZURBARÁN, Francisco de","(b. 1598, Fuente de Cantos, d. 1664, Madrid)",The House of Nazareth,c. 1630,Oil on canvas,"Museum of Art, Cleveland",https://www.wga.hu/art/z/zurbaran/2/nazareth.jpg,painting,religious,Spanish,1601-1650
50552,"ZURBARÁN, Francisco de","(b. 1598, Fuente de Cantos, d. 1664, Madrid)",Portrait of the Duke of Medinaceli,-,Oil on canvas,"Hospital of Tavera, Toledo",https://www.wga.hu/art/z/zurbaran/2/port_med.jpg,painting,portrait,Spanish,1601-1650
50553,"ZURBARÁN, Francisco de","(b. 1598, Fuente de Cantos, d. 1664, Madrid)",St Francis,c. 1660,"Oil on canvas, 65 x 53 cm","Alte Pinakothek, Munich",https://www.wga.hu/art/z/zurbaran/2/sfrancis.jpg,painting,religious,Spanish,1601-1650
50554,"ZURBARÁN, Francisco de","(b. 1598, Fuente de Cantos, d. 1664, Madrid)",Saint James of the Marches,-,"Oil on canvas, 291 x 165 cm","Museo del Prado, Madrid",https://www.wga.hu/art/z/zurbaran/2/st_james.jpg,painting,religious,Spanish,1601-1650


In [100]:
df.iloc[1] 

AUTHOR                                   AACHEN, Hans von
BORN-DIED                 (b. 1552, Köln, d. 1615, Praha)
TITLE                                            Allegory
DATE                                                 1598
TECHNIQUE                       Oil on copper, 56 x 47 cm
LOCATION                          Alte Pinakothek, Munich
URL          https://www.wga.hu/art/a/aachen/allegory.jpg
FORM                                             painting
TYPE                                         mythological
SCHOOL                                             German
TIMEFRAME                                       1601-1650
Name: 1, dtype: object

In [101]:
df["SCHOOL"].value_counts()

Italian          13637
Dutch             4374
French            4326
Flemish           3693
German            1883
Spanish           1237
English            650
Netherlandish      514
Hungarian          345
Austrian           323
American           178
Swiss              127
Danish             125
Russian            111
Belgian             83
Scottish            74
Catalan             51
Greek               47
Other               41
Bohemian            39
Swedish             37
Irish               33
Portuguese          25
Norwegian           24
Polish              18
Finnish              5
Name: SCHOOL, dtype: int64

In [99]:
df["AUTHOR"].nunique()

(3630,
 array(['Oil on canvas, 68 x 95 cm', 'Oil on copper, 56 x 47 cm',
        'Oil on canvas, 197 x 142 cm', ..., 'Oil on canvas, 121,5 x 97 cm',
        'Oil on canvas, 139 x 104 cm', 'Oil on canvas, 136,5 x 102,5 cm'],
       dtype=object))

In [ ]:
df["TECHNIQUE"].unique()

In [ ]:
df["TYPE"].value_counts()

In [ ]:
df["TIMEFRAME"].value_counts() 

In [ ]:
df_feature_encoded = df.drop(columns=["BORN-DIED", "TITLE", "DATE", "TECHNIQUE", "LOCATION", "URL", "FORM"])
df_feature_encoded

In [43]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from vincentvanbot.preprocessing.featureencoder import transformer, get_col_transformed
from vincentvanbot.preprocessing.pipeline import build_pipe

In [14]:
transformer(df)

ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('onehot',
                                 OneHotEncoder(categories='auto', drop=None,
                                               dtype=<class 'numpy.float64'>,
                                               handle_unknown='error',
                                               sparse=False),
                                 [0, 1])],
                  verbose=False)

In [61]:
df_col_encoded = get_col_transformed(transformer, df)
df_col_encoded

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
31996,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
31997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
31998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [102]:
img2 = np.array(df_col_encoded.iloc[2000:2001])
img2.shape

(1, 36)

In [93]:
df_col_encoded_reduced = df_col_encoded[:100]
df_col_encoded_reduced.shape

(100, 36)

In [95]:
#frames = [df_col_encoded_reduced, df_transformed]

#result = pd.concat(frames,  axis=1,)
#result

In [103]:
knn_model = NearestNeighbors(n_neighbors=5).fit(df_col_encoded)
ind_list = list(knn_model.kneighbors(img2,n_neighbors=5)[1][0])#[1:] # get n closest points, unpack indices to a list
df_closest_images = df.iloc[ind_list, :]
df_closest_images

,AUTHOR,BORN-DIED,TITLE,DATE,TECHNIQUE,LOCATION,URL,FORM,TYPE,SCHOOL,TIMEFRAME
991,"ANESI, Paolo","(b. 1697, Roma, d. 1773, Roma)",An Extensive Landscape,1751-55,"Oil on canvas, 130 x 97 cm",Private collection,https://www.wga.hu/art/a/anesi/landsca1.jpg,painting,landscape,Italian,1701-1750
5710,"BISON, Giuseppe Bernardino","(b. 1762, Palmanova, d. 1844, Milano)",Landscape with Travellers,-,"Tempera on parchment, 32 x 45 cm",Private collection,https://www.wga.hu/art/b/bison/landsca2.jpg,painting,landscape,Italian,1801-1850
5709,"BISON, Giuseppe Bernardino","(b. 1762, Palmanova, d. 1844, Milano)",Landscape with Soldiers on Horseback,-,"Tempera on parchment, 47 x 64 cm",Private collection,https://www.wga.hu/art/b/bison/landsca1.jpg,painting,landscape,Italian,1801-1850
8660,CANALETTO,"(b. 1697, Venezia, d. 1768, Venezia)","The Churches of the Redentore and San Giacomo,...",1747-49,"Oil on canvas, 47 x 77 cm",Private collection,https://www.wga.hu/art/c/canalett/6/canal620.jpg,painting,landscape,Italian,1701-1750
4294,"BASSANO, Francesco","(b. 1549, Bassano, d. 1592, Venezia)",Summer,-,"Oil on canvas, 97 x 127 cm","The Hermitage, St. Petersburg",https://www.wga.hu/art/b/bassano/francesc/summ...,painting,landscape,Italian,1551-1600


### PCA - check visually with different components

In [ ]:
lfw_path = "/Users/juliaschmitt/code/JulSchmitt/vincentvanbot/raw_data/images"

images = []
for root, dirnames, filenames in os.walk(lfw_path):
    for filename in fnmatch.filter(filenames, '*.jpg'):
        images.append(os.path.join(root, filename))

n = len(images)
print("loaded %d images" % n)

In [ ]:
# random selection of images

img = [Image.open(images[int(n*random.random())]) for i in range(8)]
#img[1] = np.concatenate([np.array(im.getdata()).reshape((im.size[0], im.size[1], 3)) for im in img ], axis=1)
#plt.figure(figsize=(16,2))
#plt.imshow(Image.fromarray(img.astype('uint8')))


#concat_image = np.concatenate([np.asarray(t) for t in img], axis=1)

#img = [[X[idx].reshape((w,h,3))] for idx in np.random.randint(n, size=8)]
#img = np.concatenate([np.asarray(t) for t in img], axis=1)
#plt.figure(figsize=(16,8))
#plt.imshow(Image.fromarray(img_all.astype('uint8')))

#img = img.resize((int(img.width * 100 / img.height), 100))

In [ ]:
n_components = 50
pca = PCA(n_components=n_components, svd_solver='randomized', whiten=True)
pca.fit(X)

In [ ]:
Xp = pca.transform(df)
Xp.shape

In [ ]:
Xt = pca.inverse_transform(Xp)
Xt = np.clip(Xt, 0, 255)

In [ ]:
w = h = 100

In [ ]:
test_1 = X[1].reshape((w,h,3)) #for idx in np.random.randint(n, size=8)][1]
test_2 = Xt[1].reshape((w,h,3)) #for idx in np.random.randint(n, size=8)][1]
stack_img = np.hstack((test_1, test_2))
plt.figure(figsize=(32,4))
plt.imshow(stack_img)


In [ ]:
test_x = [X[idx].reshape((w,h,3)) for idx in np.random.randint(n, size=8)][1]
plt.imshow(test_x)

In [ ]:
test_xt = [Xt[idx].reshape((w,h,3)) for idx in np.random.randint(n, size=8)][1]
plt.imshow(test_xt)

In [ ]:
npca = [100, 50, 10, 2, 1]
Xt = []
for p in npca:
    print("computing PCA for %d components"%p)
    pca = PCA(n_components=p, svd_solver='randomized', whiten=True)
    pca.fit(X)
    Xp = pca.transform(X)
    Xt.append(np.clip(pca.inverse_transform(Xp), 0, 255))

In [ ]:
idxs = np.random.randint(n, size=3)
for i,idx in enumerate(idxs):
    img_orig = X[idx].reshape((w,h,3))
    img_reconstructed = np.concatenate([Xt[p][idx].reshape((w,h,3)) for p in range(len(npca))], axis=1)
    plt.figure(figsize=(2,2))
    plt.imshow(Image.fromarray(img_orig.astype('uint8')))
    plt.figure(figsize=(16,2))
    plt.imshow(Image.fromarray(img_reconstructed.astype('uint8')))


In [ ]:
img_pairs = [[X[idx].reshape((w,h,3)), Xt[idx].reshape((w,h,3))] for idx in np.random.randint(n, size=8)]
img_all = np.concatenate([np.concatenate([p[0], p[1]]) for p in img_pairs], axis=1)
plt.figure(figsize=(32,8))
plt.imshow(Image.fromarray(img_all.astype('uint8')))
